In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.keras import Sequential
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
#from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from keras import Sequential
from keras.optimizers import Adam
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Conv2D, Flatten, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
import cv2
from PIL import Image

In [4]:
df = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/train.csv")
df_test = pd.read_csv("E:\Data Science\Kaggle Competitions dataset\Hackerearth gala\dataset/test.csv")

In [5]:
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [6]:
# True image size
IMG_WIDTH = 120
IMG_HEIGHT = 80

def preprocess_image(image, sigmaX=10):
    """
    The whole preprocessing pipeline:
    1. Read in image
    2. Apply masks
    3. Resize image to desired size
    4. Add Gaussian noise to increase Robustness
    
    :param img: A NumPy Array that will be cropped
    :param sigmaX: Value used for add GaussianBlur to the image
    
    :return: A NumPy array containing the preprocessed image
    """
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT))
    image = cv2.addWeighted (image,4, cv2.GaussianBlur(image, (0,0) ,sigmaX), -4, 128)
    return image

In [7]:
y = df["Class"]

In [8]:
from matplotlib import pyplot as plt
%matplotlib inline

In [9]:
#img = cv2.imread(r'E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/image7042.jpg')
#cv2.imshow('image',img)
#cv2.waitKey(0)

In [10]:
# MAKING SURE ALL IMAGES ARE OF SAME SIZE

#diff_shape = set()
PATH = "E:/Data Science/Kaggle Competitions dataset/Hackerearth gala/dataset/Train Images/"
#for i in range(df.shape[0]):
#    temp = PATH + str(df["Image"][i])
#    img = cv2.imread(temp)
#    diff_shape.add(img.shape)

In [11]:
BATCH_SIZE = 4
# resizing the image to the size of efficientnet
#IMG_WIDTH = 456
#IMG_HEIGHT = 456

# Add Image augmentation to our generator
train_datagen = ImageDataGenerator(rotation_range=360,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2,
                                   preprocessing_function=preprocess_image, 
                                   rescale=1 / 128.)
train_generator = train_datagen.flow_from_dataframe(df, 
                                                    x_col='Image', 
                                                    y_col='Class',
                                                    directory = PATH,
                                                    target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical', 
                                                    subset='training')
val_generator = train_datagen.flow_from_dataframe(df, 
                                                  x_col='Image', 
                                                  y_col='Class',
                                                  directory = PATH,
                                                  target_size=(IMG_WIDTH, IMG_HEIGHT),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical',
                                                  subset='validation')

Found 4787 validated image filenames belonging to 4 classes.
Found 1196 validated image filenames belonging to 4 classes.


In [12]:
## seems like efficient net is too large
# can try to freeze some layers


#from keras_efficientnets import EfficientNetB5
#effnet = EfficientNetB5(input_shape=(456,456,3),
#                        weights='imagenet',
#                        include_top=False)

In [13]:
#effnet.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 456, 456, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 228, 228, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 228, 228, 48) 192         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 228, 228, 48) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [ ]:
from keras.regularizers import l2
l2 = l2(0.01)

In [14]:
def build_model():
    """
    A custom implementation of EfficientNetB5
    for the APTOS 2019 competition
    (Regression)
    """
    model = Sequential()
    
    model.add()
    model.add(Conv2D(filters = 30, kernel_size = (5,5),padding = 'Same', strides = 1, 
                 activation ='relu', input_shape = (IMG_WIDTH,IMG_HEIGHT,3),kernel_initializer = "lecun_uniform",kernel_regularizer = l2))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5))
    model.add(Dense(20, activation="relu"))
    model.add(Dense(4, activation="softmax"))
    model.compile(loss='categorical_crossentropy',
                  optimizer="adam", 
                  metrics=['categorical_crossentropy', 'mse'])
    print(model.summary())
    return model

# Initialize model
model = build_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 15, 15, 2048)      28513520  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                40980     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 84        
Total params: 28,554,584
Trainable params: 28,381,848
Non-trainable params: 172,736
_________________________________________________________________
None


In [15]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=12)
rlr = ReduceLROnPlateau(monitor='val_loss', 
                        factor=0.5, 
                        patience=4, 
                        verbose=1, 
                        mode='auto', 
                        epsilon=0.00001)

# Begin training
model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=35,
                    validation_data=val_generator,
                    validation_steps = val_generator.samples // BATCH_SIZE,
                    callbacks=[es, rlr])


C:\Users\ishtd\Anaconda3\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:998: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/35


ResourceExhaustedError:  OOM when allocating tensor with shape[4,1056,29,29] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node gradients/model_1/batch_normalization_78/cond_grad/If/then/_3793/zeros_like}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_138357]

Function call stack:
keras_scratch_graph
